In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

stemmer = LancasterStemmer()

# Load intents data from a JSON file
with open("intents.json") as file:
    data = json.load(file)

try:
    # Try to load preprocessed data from a pickle file
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    # Preprocess the data if the pickle file doesn't exist
    words = []
    labels = []
    docs_x = []
    docs_y = []

    # Iterate through intents and patterns to collect words and labels
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            # Tokenize and stem each word
            wrds = nltk.word_tokenize(pattern) 
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])
            
    # Stem and lowercase words, removing duplicate entries
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]
    
    # Create training data and output based on the collected words and labels
    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)
    
    # Save the preprocessed data to a pickle file for future use
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)
        
# Reset TensorFlow graph
tensorflow.compat.v1.reset_default_graph()

# Define the neural network architecture
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    # Try to load the trained model
    model.load("model.tflearn")
except:
    # Train the model if the saved model doesn't exist
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

# Define a function to convert input sentence into a bag of words representation
def bag_of_words(sentence, words):
    bag = [0 for _ in range(len(words))]
    
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    
    for se in sentence_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return numpy.array(bag)


# Define the chat function for user interaction
def chat():
    print("Start chating with the bot...(type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        
        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg["tag"] == tag:
                responses = tg["responses"]
        
        print(random.choice(responses))
       
chat()

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from c:\Users\Hp\Documents\AI\Chatbot\model.tflearn
Start chating with the bot...(type quit to stop)
Hello!
Hello friends ;)
My pleasure
Our company is situated in 'street name', Addis Ababa
We are on 'street name', Addis Ababa
